In [1]:
import pandas as pd
from m2p import PolyMaker
from polyid import generate_hash
from fingerprints import HierarchticalFingerprints

pm = PolyMaker()
hfp = HierarchticalFingerprints()

%load_ext autoreload
%autoreload 2

2023-03-31 10:14:16.149791: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Generate Structures

In [ ]:
# import data
data = pd.read_csv('../data/example_monomer_smiles.csv',index_col=0)

# generate monomers tuple
data['monomers'] = data.smiles_monomer.apply(pm.get_monomers)

# # generate polymer structures with DP=8 and 2 replicate stuctures
data = pm.thermoplastic(data.sample(5),DP=8,mechanism='all',replicate_structures=1)

# remove any polymers which may have resulted in an error during reaction
dferror = data[data.smiles_polymer.str.contains('ERR')]
data = data[~data.smiles_polymer.str.contains('ERR')]

# hash columns for tracking predictions and structures
data = generate_hash(data,hash_cols=['monomers','replicate_structure'])
data = generate_hash(data.reset_index(),hash_cols=['monomers'])

data.head()

# Generate Atomic and Molecular/Morphological Fingerprints

In [ ]:
hfp.gen_fp_atomic_molecular_morphological(data)

print('Atomic FPs\n{}\n'.format(hfp.df_atomic.to_markdown()))
print('Molecular and Morpholotical (mordred) FPs\n{}\n'.format(hfp.df_molecular_morphological.to_markdown()))
print('Atomic, Molecular/Morpholotical (mordred) FPs\n{}\n'.format(hfp.df_atomic_molecular_morphological.to_markdown()))